In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy as sc

In [87]:
class Station():
    def __init__(self, station_name, num_bikes, num_docks, arr_stream):
        
        #Initialization of Station Class:
        #station_name: Name of station (string)
        #num_bikes: number of bikes at the station initially (int)
        #num_docks: number of docks at the station initially (int)
        #arr_stream: data stream containing our arrivals to the station (Pandas DataFrame)
        
        self.station_name = station_name
        self.num_bikes = num_bikes
        self.num_docks = num_docks
        self.arr_stream = arr_stream
        self.simulate(num_bikes, num_docks, arr_stream)
        print("Station " + station_name + " has been initialized")
        
    def simulate(self, num_bikes, num_docks, arr_stream):
        num_bikes = num_bikes #initial number of bikes
        num_docks = num_docks #initial number of docks
        arr_stream = arr_stream
        self.wait_time_arr = [] #array of wait times for customers
        self.bike_num_arr = [] #array of bike numbers after each "transaction"
        self.dock_num_arr = [] #array of dock numbers after each "transaction"
        
        for i in range(len(arr_stream)): #For all the values in our data stream            
            if arr_stream.iloc[i, :]["Type_Of_Customer"] == "Returning": #if customer is returning a bike                
                if num_docks > 0: #If there is an available dock                    
                    wait_time = 0 #instant service (PLAY WITH THIS. COULD ADD DELAY)
                    
                else: #If there is no available docks                    
                    #Customer must wait until another customer comes and borrows a bike
                    remaining_df = arr_stream.iloc[i+1: len(arr_stream), :] #Look in the remaining df
                    next_borrow_df = remaining_df.where(remaining_df["Type_Of_Customer"] == "Borrowing").dropna() #Find the people borrowing
                    next_avail_dock_time = min(next_borrow_df["Arrival_Time"]) #Find the minimum time
                    wait_time = next_avail_dock_time - arr_stream.iloc[i, :]["Arrival_Time"] #Wait time is next dock time - arrival_time of current customer
                
                num_bikes = max(0, num_bikes + 1) #returning a bike means one more bike
                num_docks = max(0, num_docks - 1) #returning a bike means one less dock
                
            else: #Else customer is borrowing (customers can only be returning or borrowing)
                if num_bikes  > 0: #If there is an available bike                   
                    wait_time = 0 #instant service (PLAY WITH THIS. COULD ADD DELAY)
                    
                else: #If there is no available bikes                   
                    #Customer must wait until another customer comes and returns a bike
                    remaining_df = arr_stream.iloc[i+1: len(arr_stream), :] #Look in the remaining df
                    next_returning_df = remaining_df.where(remaining_df["Type_Of_Customer"] == "Returning").dropna() #Find the people returning
                    next_avail_bike_time = min(next_returning_df["Arrival_Time"]) #Find the minimum time
                    wait_time = next_avail_bike_time - arr_stream.iloc[i, :] ["Arrival_Time"]#Wait time is next dock time - arrival_time of current customer
                
                num_bikes = max(0, num_bikes - 1) #borrowing a bike means one less bike
                num_docks = max(0, num_docks + 1) #borrowing a bike means one more dock
                
            #Append information to our arrays:
            self.wait_time_arr.append(wait_time)
            self.bike_num_arr.append(num_bikes)
            self.dock_num_arr.append(num_docks)
            
    def get_wait_time(self):
        return self.wait_time_arr
    
    def get_bike_num(self):
        return self.bike_num_arr
    
    def get_dock_num(self):
        return self.dock_num_arr


In [90]:
test_df = pd.DataFrame({"Arrival_Time": [0],
                        "Type_Of_Customer": ["Borrowing"],
                        "Station_From": ["Joe"]
                       })
test_station = Station("Bob", 0, 5, test_df)

ValueError: min() arg is an empty sequence

In [89]:
test_station.get_wait_time()

[1, 0]

In [92]:
test_station.get_bike_num()

[0, 1]

In [93]:
test_station.get_dock_num()

[6, 5]

In [84]:
a

[0, 1, 2, 3]